In [1]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import re
import os
from os import mkdir
from os.path import isdir, isfile

import warnings
warnings.filterwarnings("ignore")

In [3]:
import tika
tika.initVM()

In [4]:
from tika import parser

def get_document_text(filepath):
    doc_type = filepath.split('.')[-1]
    if doc_type in ['doc', 'rtf', 'pdf', 'docx']:
        text = parser.from_file(filepath)['content']
    else:
        print(f'File extention must be either pdf, rtf, doc or docx, got {doc_type}')
    return text.strip()

In [5]:
URL_TEMPLATE = "https://amulex.ru/docs/"
docs = requests.get(URL_TEMPLATE, headers={'User-Agent': 'Custom'})
print(docs.status_code)

200


In [8]:
'''
Функция make_dir скачивает pdf-файлы с сайта https://amulex.ru и сохраняет их по папкам, разделяя на классы

На вход подается ссылка на категорию договоров (category_contract_url), название класса договора (folder_name)
1. Переходит в категорию category_contract_url
2. Проходя по всей категории, скачивает договоры, переименовывает их с использованием _
3. Сохраняет скаченные файлы в папку с нужным классом

Параметры:
1. category_contract_url:
    Ссылка на категорию договоров на сайте. Договоры определенного класса могут быть из разных категорий
2. folder_name:
    Название класса договора из списка: Договоры аренды, Договоры купли-продажи, Договоры оказания услуг, Договоры подряда, Договоры поставки
'''
def make_dir(category_contract_url, folder_name, folder_path='../data/parsed_data/'):
    category_cite = requests.get('https://amulex.ru' + category_contract_url, headers={'User-Agent': 'Custom'})
    categ = bs(category_cite.text, "html.parser")
    contract_url = categ.find_all('li',class_='items__item', id = False)
    contract_names = categ.find_all('h5',class_="items__item-title")
    for i in range(len(contract_url)):
        name_words = contract_names[i].text.split()
        name = ''
        for k in name_words:
            name += ''.join(e for e in k if e.isalnum())
            if k != name_words[-1]:
                name += '_'
        contract_cite = requests.get('https://amulex.ru' + contract_url[i].a['href'], headers={'User-Agent': 'Custom'})
        pars = bs(contract_cite.text, "html.parser")
        contract_download_url = pars.find('div',class_='download__dropdown js-dropdown')
        x = requests.get(contract_download_url.a['href'])
        if isfile(folder_path + folder_name + '/' + name + '.pdf') == False:
            with open(folder_path + folder_name + '/' + name + '.pdf', 'wb') as f:
                f.write(x.content)

In [27]:
soup = bs(docs.text, "html.parser")
category_contract_url = soup.find_all('a', class_='category__item-link')
folder_names = ['Договоры аренды', 'Договоры купли-продажи', 'Договоры оказания услуг', 'Договоры подряда', 'Договоры поставки']
for i in folder_names:
    mkdir('/home/jovyan/work/data/parsed_data/' + str(i))
for j in range(len(category_contract_url)):
    s=category_contract_url[j].h2.text
    re.sub(r"\s+", "", s)
    if 'аренд' in s:
        make_dir(category_contract_url[j]['href'], folder_names[0])
    elif 'купл' in s:
        make_dir(category_contract_url[j]['href'], folder_names[1])
    elif 'услуг' in s:
        make_dir(category_contract_url[j]['href'], folder_names[2])
    elif 'подряд' in s:
        make_dir(category_contract_url[j]['href'], folder_names[3])
    elif 'постав' in s:
        make_dir(category_contract_url[j]['href'], folder_names[4])

time: 55min 19s (started: 2023-01-25 17:56:53 +03:00)


In [9]:
path = '/home/jovyan/work/data/parsed_data/'
docs = pd.DataFrame(columns = ['text','filename','target'])
count = 0
folder_names = ['Договоры аренды', 'Договоры купли-продажи', 'Договоры оказания услуг', 'Договоры подряда', 'Договоры поставки']

for fold_name in folder_names:
    folder_path = os.chdir(path + fold_name)
    for name in os.listdir(folder_path):
        text = get_document_text(name)
        new_row = {'text': text,
                   'filename': name,
                   'target': fold_name}
        docs = docs.append(new_row, ignore_index=True)

2023-01-30 13:05:06,557 [MainThread  ] [WARNI]  Failed to see startup log message; retrying...


In [11]:
docs.to_csv(path + 'parsed_data.csv', sep='\t',index=False, encoding = 'utf-8')